# 04. Model 2: Personalized Advice Generation

## Objective
Menghasilkan saran pembelajaran yang **personal, empatik, dan actionable** dengan menggabungkan:
- **Model 1 Output**: Cluster Label (Persona)
- **Model 3 Output**: Pace Comparison
- **Raw Data**: Exam scores, stuck points

## Approach
**Prompt Engineering with Gemini API** (Free Tier)
- ✅ No fine-tuning needed (dataset kecil, 31 users)
- ✅ Context-aware templates untuk 5 persona
- ✅ Production-ready

## 5 Persona
1. **The Sprinter**: Fast & High Score
2. **The Deep Diver**: Slow but Good Score
3. **The Night Owl**: Active 19-24 (malam)
4. **The Struggler**: Low score, banyak fail
5. **The Consistent**: Belajar rutin/stabil

In [15]:
# === 1. SETUP ===
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Paths
PROCESSED_DIR = '../data/processed'
MODELS_DIR = '../models'
os.makedirs(MODELS_DIR, exist_ok=True)

print("✅ Environment Ready!")
print(f"📁 Processed data: {PROCESSED_DIR}")
print(f"💾 Models output: {MODELS_DIR}")

✅ Environment Ready!
📁 Processed data: ../data/processed
💾 Models output: ../models


---
# STEP 1: INSTALL & CONFIGURE GEMINI API

In [2]:
# Install Gemini SDK (jika belum)
!pip install -q google-generativeai

print("✅ Gemini SDK installed!")

✅ Gemini SDK installed!


In [ ]:
import google.generativeai as genai

# ===================================
# IMPORTANT: GANTI DENGAN API KEY ANDA!
# Dapatkan gratis di: https://makersuite.google.com/app/apikey
# ===================================

API_KEY = "API_KEY"

# Configure API
genai.configure(api_key=API_KEY)

# Test connection
model = genai.GenerativeModel('gemini-2.0-flash')
test_response = model.generate_content("Say 'Hello' in Indonesian")
print("🔗 Gemini API Connected!")
print(f"Test response: {test_response.text}")

🔗 Gemini API Connected!
Test response: The most common way to say "Hello" in Indonesian is:

**Halo**

You can also use:

*   **Selamat pagi** (Good morning)
*   **Selamat siang** (Good day/afternoon - usually used from 11:00 am to 3:00 pm)
*   **Selamat sore** (Good afternoon/evening - usually used from 3:00 pm to sunset)
*   **Selamat malam** (Good evening/night)


---
# STEP 2: LOAD DATA CONTEXT

File yang dibutuhkan:
1. `advice_context.csv` - Context data (nama, persona, pace)
2. `clustering_results.csv` - Cluster labels (dari Model 1)
3. `pace_features.csv` - Pace data (dari Model 3)

In [17]:
print("="*60)
print("LOADING CONTEXT DATA")
print("="*60)

# Load main context file
df_advice = pd.read_csv(os.path.join(PROCESSED_DIR, 'advice_context.csv'))
print(f"\n📂 Advice context loaded: {df_advice.shape}")
print(f"Columns: {list(df_advice.columns)}")

# Load clustering results (untuk cluster_label)
df_clusters = pd.read_csv(os.path.join(PROCESSED_DIR, 'clustering_results.csv'))
print(f"\n📂 Clustering results loaded: {df_clusters.shape}")

# Merge cluster labels jika belum ada di advice_context
if 'cluster_label' not in df_advice.columns:
    df_advice = df_advice.merge(
        df_clusters[['developer_id', 'journey_id', 'cluster_label']], 
        on=['developer_id', 'journey_id'], 
        how='left'
    )
    print("✅ Cluster labels merged!")

print(f"\n📊 Final dataset shape: {df_advice.shape}")
print(f"✅ Ready for advice generation!")

LOADING CONTEXT DATA

📂 Advice context loaded: (2008, 18)
Columns: ['developer_id', 'journey_id', 'name', 'avg_study_hour', 'study_time_slot', 'avg_exam_score', 'exam_fail_count', 'avg_submission_rating', 'submission_fail_count', 'completion_speed', 'performance_level', 'struggle_score', 'speed_category', 'id', 'display_name', 'stuck_tutorial_id', 'pace_insight', 'cluster_label']

📂 Clustering results loaded: (1907, 9)

📊 Final dataset shape: (2008, 18)
✅ Ready for advice generation!


In [11]:
# Preview data
print("\n🔍 Sample data:")
display(df_advice[[
    'developer_id', 'journey_id', 'name', 'cluster_label', 
    'avg_exam_score', 'completion_speed'
]].head())

print("\n📊 Cluster distribution:")
print(df_advice['cluster_label'].value_counts())


🔍 Sample data:


,developer_id,journey_id,name,cluster_label,avg_exam_score,completion_speed
0,3390,14,Belajar Fundamental Aplikasi Android,The Struggler,62.500000,0.521429
1,3390,32,Belajar Membangun LINE Chatbot,The Consistent,83.846154,0.560000
2,3390,47,Menjadi Game Developer Expert,The Struggler,83.846154,0.578571
3,3390,51,Belajar Membuat Aplikasi Android untuk Pemula,The Consistent,65.000000,0.560000
4,3390,55,Kotlin Android Developer Expert,The Struggler,83.846154,0.600000



📊 Cluster distribution:
cluster_label
The Sprinter      762
The Struggler     528
The Night Owl     375
The Consistent    343
Name: count, dtype: int64


---
# STEP 3: PROMPT ENGINEERING

## Design Strategy:
1. **Base Template**: Struktur umum untuk semua persona
2. **Persona-Specific Context**: Variasi saran berdasarkan cluster
3. **Dynamic Variables**: Name, score, pace, stuck_point

In [19]:
# === PROMPT TEMPLATES ===

# Base template (untuk semua persona)
BASE_TEMPLATE = """
Kamu adalah AI Learning Coach untuk platform pembelajaran programming Dicoding.

DATA SISWA:
- Nama Siswa: {student_name}
- Kursus: {course_name}
- Persona Belajar: {persona}
- Pace Belajar: {pace_info}
- Rata-rata Score Kuis: {avg_score:.1f}
- Status: {status_info}

TUGAS:
Berikan saran pembelajaran yang:
1. **Personal**: Panggil nama siswa
2. **Empatik**: Sesuai dengan persona DAN pace belajar mereka
3. **Actionable**: Berikan 1-2 saran konkret yang bisa langsung dipraktikkan

{persona_context}

{pace_context}

FORMAT OUTPUT:
- 1-2 paragraf dalam Bahasa Indonesia
- Tone: Sopan, memotivasi, dan suportif
- Hindari jargon teknis yang rumit
- Gabungkan insight dari persona DAN pace belajar
"""

# Persona-specific context
PERSONA_CONTEXTS = {
    'The Sprinter': """
CONTEXT PERSONA:
Siswa ini adalah Fast Learner yang menyelesaikan materi dengan CEPAT dan nilai TINGGI.
Kriteria: completion_speed rendah + avg_exam_score tinggi

FOKUS SARAN PERSONA:
- Apresiasi kecepatan dan kemampuannya
- Sarankan tantangan lebih tinggi (advanced topics, project-based learning)
- Dorong sharing knowledge dengan teman
""",
    
    'The Deep Diver': """
CONTEXT PERSONA:
Siswa ini belajar dengan tempo LAMBAT tapi MENDALAM (reflective learner).
Kriteria: completion_speed tinggi + avg_exam_score tinggi (lambat tapi nilai bagus)

FOKUS SARAN PERSONA:
- Apresiasi ketelitian dan pemahaman mendalamnya
- Sarankan tetap pertahankan kualitas, jangan terburu-buru
- Dorong dokumentasi proses belajarnya
""",
    
    'The Night Owl': """
CONTEXT PERSONA:
Siswa ini aktif belajar di MALAM HARI (jam 19.00 - 24.00).
Kriteria: avg_study_hour >= 19

FOKUS SARAN PERSONA:
- Akui jadwal belajar malamnya
- Sarankan optimasi jadwal (breaks, kesehatan mata, teknik pomodoro)
- Ingatkan pentingnya istirahat cukup dan jaga kesehatan
""",
    
    'The Struggler': """
CONTEXT PERSONA:
Siswa ini mengalami KESULITAN (nilai rendah, banyak submission/kuis gagal).
Kriteria: avg_exam_score rendah + submission_fail_rate tinggi + retry_count tinggi

FOKUS SARAN PERSONA:
- Berikan motivasi dan empati (jangan menyerah!)
- Sarankan resources tambahan (review materi, forum diskusi, mentor)
- Dorong untuk tidak menyerah, satu langkah kecil dulu
- Fokus pada pemahaman konsep dasar
""",
    
    'The Consistent': """
CONTEXT PERSONA:
Siswa ini belajar dengan RUTIN dan KONSISTEN setiap hari/minggu.
Kriteria: study_consistency_std rendah (stabil)

FOKUS SARAN PERSONA:
- Apresiasi konsistensi dan disiplinnya
- Sarankan tetap pertahankan momentum
- Dorong set target jangka panjang
- Ajarkan teknik time-blocking untuk efisiensi
"""
}

PACE_CONTEXTS = {
    'fast learner': """
CONTEXT PACE:
Berdasarkan analisis pace, siswa ini termasuk FAST LEARNER:
- Menyelesaikan banyak materi dalam waktu singkat (>5 materi/hari)
- Kecepatan belajar di atas rata-rata

FOKUS SARAN PACE:
- Pastikan pemahaman tidak dikorbankan demi kecepatan
- Dorong untuk eksplorasi topik advanced
""",

    'consistent learner': """
CONTEXT PACE:
Berdasarkan analisis pace, siswa ini termasuk CONSISTENT LEARNER:
- Belajar secara rutin dan teratur
- CV (Coefficient of Variation) mingguan rendah

FOKUS SARAN PACE:
- Pertahankan ritme belajar yang sudah baik
- Optimalkan waktu belajar dengan teknik spaced repetition
""",

    'reflective learner': """
CONTEXT PACE:
Berdasarkan analisis pace, siswa ini termasuk REFLECTIVE LEARNER:
- Menghabiskan waktu lebih untuk memahami materi secara mendalam
- Sering review materi yang sudah dipelajari

FOKUS SARAN PACE:
- Apresiasi kedalaman pemahaman
- Dorong untuk mencatat insight dan membuat rangkuman
"""
}   

print("✅ Prompt templates defined!")
print(f"📝 Base template length: {len(BASE_TEMPLATE)} chars")
print(f"🎭 Personas covered: {list(PERSONA_CONTEXTS.keys())}")
print(f"PACE Personas covered: {list(PACE_CONTEXTS.keys())}")

✅ Prompt templates defined!
📝 Base template length: 698 chars
🎭 Personas covered: ['The Sprinter', 'The Deep Diver', 'The Night Owl', 'The Struggler', 'The Consistent']
PACE Personas covered: ['fast learner', 'consistent learner', 'reflective learner']


---
# STEP 4: ADVICE GENERATION FUNCTION

In [20]:
def generate_advice(row, model, verbose=False):
    """
    Generate personalized advice untuk satu user-journey pair.
    IMPROVED VERSION dengan integrasi Model 1 (Persona) dan Model 3 (Pace)
    
    Args:
        row: DataFrame row dengan info user (harus memiliki kolom yang sesuai)
        model: Gemini model instance
        verbose: Print progress atau tidak
    
    Returns:
        str: Generated advice text
    """
    import time
    
    try:
        # Mengambil nama siswa dari display_name
        student_name = row.get('display_name', 'Siswa')
        if not student_name or student_name == 'Siswa':
            student_name = f"Siswa ID-{row.get('developer_id', 'unknown')}"
        
        # Nama kursus terpisah
        course_name = row.get('name', 'Kursus')
        
        # Persona dari Model 1 (clustering)
        persona = row.get('cluster_label', 'Unknown')
        
        # Pace label dari Model 3 (pace analysis)
        pace_label = row.get('pace_label', None)
        
        # Metrics
        avg_score = row.get('avg_exam_score', 0)
        completion_speed = row.get('completion_speed', 1.0)
        
        # BUILD PACE INFO
        if pace_label:
            pace_info = f"{pace_label.title()}"
        elif completion_speed < 0.7:
            pace_info = f"Lebih cepat {(1-completion_speed)*100:.0f}% dari rata-rata"
        elif completion_speed > 1.3:
            pace_info = f"Lebih lambat {(completion_speed-1)*100:.0f}% dari rata-rata"
        else:
            pace_info = "Sesuai dengan rata-rata kelas"
        
        # BUILD STATUS INFO
        if avg_score >= 85:
            status_info = "Performa sangat baik ⭐"
        elif avg_score >= 70:
            status_info = "Performa cukup baik 👍"
        elif avg_score >= 50:
            status_info = "Perlu peningkatan 📈"
        else:
            status_info = "Butuh bantuan ekstra 💪"
        
        persona_context = PERSONA_CONTEXTS.get(persona, "")
        pace_context = PACE_CONTEXTS.get(pace_label, "") if pace_label else ""
        
        # BUILD FINAL PROMPT
        prompt = BASE_TEMPLATE.format(
            student_name=student_name,
            course_name=course_name,
            persona=persona,
            pace_info=pace_info,
            avg_score=avg_score,
            status_info=status_info,
            persona_context=persona_context,
            pace_context=pace_context
        )
        
        if verbose:
            print(f"\n🤖 Generating advice for: {student_name}")
            print(f"   Course: {course_name}")
            print(f"   Persona: {persona}")
            print(f"   Pace: {pace_label or 'N/A'}")
            print(f"   Score: {avg_score:.1f}")
        
        # CALL GEMINI API
        response = model.generate_content(prompt)
        advice_text = response.text.strip()
        
        if verbose:
            print(f"   ✅ Generated {len(advice_text)} chars")
        
        return advice_text
        
    except Exception as e:
        print(f"❌ Error for {row.get('developer_id', 'unknown')}: {str(e)}")
        return f"[Error generating advice: {str(e)}]"

print("✅ Advice generation function ready!")

✅ Advice generation function ready!


---
# STEP 5: TEST GENERATION (SAMPLE)

Test dengan 3 sample users (berbeda persona) sebelum batch generation.

In [21]:
print("="*60)
print("TESTING ADVICE GENERATION (3 SAMPLES)")
print("="*60)

# Pilih 3 sample dari persona berbeda
sample_personas = df_advice['cluster_label'].unique()[:3]
samples = []

for persona in sample_personas:
    sample = df_advice[df_advice['cluster_label'] == persona].iloc[0]
    samples.append(sample)

print(f"\n📋 Testing {len(samples)} samples...\n")

# Generate untuk setiap sample
for idx, sample in enumerate(samples, 1):
    print(f"\n{'='*60}")
    print(f"SAMPLE {idx}: {sample['cluster_label']}")
    print(f"{'='*60}")
    print(f"Name: {sample.get('display_name', 'N/A')}")
    print(f"Score: {sample.get('avg_exam_score', 0):.1f}")
    print(f"Speed: {sample.get('completion_speed', 1.0):.2f}x")
    
    advice = generate_advice(sample, model, verbose=True)
    
    print(f"\n📝 GENERATED ADVICE:\n")
    print(advice)
    print(f"\n(Length: {len(advice)} chars)")
    
    # Rate limiting (to avoid API quota)
    time.sleep(2)

print(f"\n\n✅ Test generation completed!")

TESTING ADVICE GENERATION (3 SAMPLES)

📋 Testing 3 samples...


SAMPLE 1: The Struggler
Name: istiabudi73
Score: 62.5
Speed: 0.52x

🤖 Generating advice for: istiabudi73
   Course: Belajar Fundamental Aplikasi Android
   Persona: The Struggler
   Pace: N/A
   Score: 62.5
   ✅ Generated 869 chars

📝 GENERATED ADVICE:

Halo Istiabudi73! Semangat terus ya dalam belajar Fundamental Aplikasi Android. Saya lihat kamu sudah melaju 48% lebih cepat dari rata-rata, itu keren sekali! Walaupun nilai kuis kamu perlu ditingkatkan, jangan khawatir, itu hal yang wajar dalam proses belajar. Ingat, yang terpenting adalah pemahaman konsep dasarnya.

Karena kamu merasa sedikit kesulitan, saya sarankan untuk coba meluangkan waktu sedikit lebih banyak untuk me-review kembali materi-materi yang kurang dipahami. Jangan ragu untuk membaca ulang modul-modul sebelumnya, atau bahkan menonton kembali video pembelajarannya. Selain itu, manfaatkan juga forum diskusi yang ada di Dicoding. Di sana kamu bisa bertanya, b

In [ ]:
sample_row = df_advice.iloc[0].to_dict()
print("\\n" + "="*50)
print("SAMPLE ADVICE:")
print("="*50)
print(generate_advice(sample_row, model))

\n==================================================
SAMPLE ADVICE:
Halo Istiabudi73! Saya melihat kamu sedang berjuang di kursus Belajar Fundamental Aplikasi Android, dan saya mengerti bahwa ini mungkin terasa menantang. Jangan khawatir, kamu tidak sendirian! Meskipun nilaimu saat ini menunjukkan perlu peningkatan, dan dengan pace belajarmu yang lebih cepat dari rata-rata, ini menunjukkan bahwa kamu memiliki potensi besar untuk sukses. Jangan menyerah ya!

Mengingat kamu adalah seorang "The Struggler" yang mungkin merasa kesulitan dengan materi, dan dengan kecepatan belajarmu yang cukup tinggi, saran saya adalah: coba luangkan waktu untuk benar-benar memahami konsep dasar sebelum lanjut ke materi berikutnya. Mungkin kamu bisa kembali meninjau materi-materi awal, terutama yang terasa sulit. Jangan ragu untuk memanfaatkan forum diskusi di Dicoding untuk bertanya jika ada hal yang kurang jelas. Selain itu, pertimbangkan untuk mencari sumber belajar lain seperti video tutorial di YouTube 

: 